In [0]:
# !pip install tensorflow 
# TensorFlow and tf.keras
# its just working with tensorflow 1.13.1, with others has problem
import tensorflow as tf
print(tf.__version__)


1.14.0


**- goolge Colab Configuration**

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


**- Setting root address to project in Google Drive**

In [0]:
!ls "/content/gdrive/My Drive/NLPHW3"
root_path = '/content/gdrive/My Drive/NLPHW3'

 allWordsSensesembeddings.vec
 BackUpCodes
'Basic BDLSTM '
 BASIC_BDLSTM_Tagger.ipynb
 Basic_Tagger_Prediction.ipynb
 Data_Prepare_PKL_Format.ipynb
 Evaluation_Datasets
'LEX BDLSTM'
 LEX_BDLSTM_Tagger.ipynb
 Lex_Tagger_Prediction.ipynb
 Mask_LSTM_Model1.ipynb
 MFS_Tagger.ipynb
'POS BDLSTM'
 POS_BDLSTM_Tagger.ipynb
'POS_LEX BDLSTM'
 POS_LEX_BDLSTM_models_48.75846501128668
 POS_LEX_BDLSTM_SenseEmbed_models_47.40406320541761
 POS_LEX_BDLSTM_Tagger.ipynb
 POS_LEX_Tagger_Prediction.ipynb
'POS_LEX_WNDOMAIN BDLSTM '
 POS_LEX_WnDomain_BDLSTM_Tagger.ipynb
 POS_LEX_WnDomain_models_47.629796839729124
 POS_LEX_WnDomain_Tagger_Prediction.ipynb
 POS_Tagger_Prediction.ipynb
'POS_WNDOMAIN BDLSTM'
 POS_WnDomain_BDLSTM_Tagger.ipynb
 POS_WnDomain_SenseEmbed_models_48.53273137697517
 POS_WnDomain_Tagger_Prediction.ipynb
 resources
 semcor
 semcor+omsti
 Training_Corpora
'WNDOMAIN BDLSTM'
 WnDomain_BDLSTM_Tagger.ipynb
 WnDomain_Tagger_Prediction.ipynb
 WSD_Evaluation_Framework


In [0]:
# Imports
from collections import Counter
import codecs
import xml.etree.cElementTree as etree
import pickle
import nltk
nltk.download('wordnet')
import numpy as np
import copy
from nltk.corpus import wordnet as wn


# Added Chars to dictionaries
UNK = "<UNK>"
PAD = "<PAD>"


RESOURCE_PATH = root_path+'/resources/'

MAP_WN2BN_FILE = RESOURCE_PATH + 'babelnet2wordnet.tsv'
MAP_BN2WNDOMAIN_FILE = RESOURCE_PATH + 'babelnet2wndomains.tsv'
MAP_BN2LEXNAMES_FILE = RESOURCE_PATH + 'babelnet2lexnames.tsv'


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
all_dict_data = pickle.load(open(root_path+'/semcor/semcor/all_dict_data.pkl', "rb"))

RawWords2id = all_dict_data["RawWords2id"]
SenseWords2id = all_dict_data["SenseWords2id"]
pos2id = all_dict_data["pos2id"]
lex2id = all_dict_data["lex2id"]
wnDomain2id = all_dict_data["wnDomain2id"]

print(len(pos2id))
print(len(SenseWords2id))

12
60919


In [0]:
# ===-----------------------------------------------------------------------===
# Trainig Section
# ===-----------------------------------------------------------------------===

# function for add padding for train_y in every batch based on maximum length of sentence in that batch
# this method boost the performance of running.
def padding(X,padding_word):
	max_len = 0
	for x in X:
		if len(x) > max_len:
			max_len = len(x)
	padded_X = np.ones((len(X), max_len), dtype=np.int32) * padding_word
	for i in range(len(X)):
		for j in range(len(X[i])):
			padded_X[i, j] = X[i][j]
	return padded_X


# padding function for train_x
def padding3(X,padding_word):
	max_len = 0
	for x in X:
		if len(x) > max_len:
			max_len = len(x)
	padded_X = np.ones((len(X), max_len), dtype=np.int32) * padding_word
	for i in range(len(X)):
		for j in range(len(X[i])):
			padded_X[i, j] = X[i][j]
	return padded_X

# function for calculating umber of hits and number of all predictions.
def number_of_batch_hits(y_batch_pred,y_batch_true):
    num_hits = 0
    num_all_chars = 0    
    for i in range(len(y_batch_true)):
        for j in range(len(y_batch_true[i])):
            num_all_chars = num_all_chars+1
            if y_batch_pred[i][j] == y_batch_true[i][j]:
                num_hits = num_hits+1    
    return num_hits, num_all_chars

# function for calculating umber of hits and number of all predictions.
def number_of_batch_sense_hits(y_batch_pred,y_batch_true):
    num_hits = 0
    num_all_chars = 0    
    for i in range(len(y_batch_true)):
        for j in range(len(y_batch_true[i])):
            if id2Sensewords[y_batch_true[i][j]].startswith('bn'):
                num_all_chars = num_all_chars+1
                if y_batch_pred[i][j] == y_batch_true[i][j]:
                    num_hits = num_hits+1    
    return num_hits, num_all_chars

# ----------------- Add Summary Function ------------------------------------------
def add_summary(writer, name, value, global_step):
    summary = tf.Summary(value=[tf.Summary.Value(tag=name, simple_value=value)])
    writer.add_summary(summary, global_step=global_step)                                 
    

## ----------------------------------------------------------------------------
def mask_lemmaToSenses_batch(x_Raw_Sent_batch, y_allSenses_Sent_batch, y_allSenses_Sent_id_batch, SenseWords2id):
     
    max_len = 0    
    for inp in x_Raw_Sent_batch:
        if len(inp) > max_len:
            max_len = len(inp)
            
    mask_lemma2sense_batch = [0]*np.ones(shape = (len(x_Raw_Sent_batch), max_len, len(SenseWords2id)))
            
    for i in range(len(x_Raw_Sent_batch)):
        for j in range(len(x_Raw_Sent_batch[i])):
            for k in range(len(y_allSenses_Sent_batch[i][j])):            
#                 if SenseWords2id.get(y_allSenses_Sent_batch[i][j][k]) is not None:
                mask_lemma2sense_batch[i][j][y_allSenses_Sent_id_batch[i][j][k]] = 1    
#                    mask_lemma2sense_batch[rawWords_batch2id[x_Raw_Sent_batch[i][j]]][y_allSenses_Sent_id_batch[i][j][k]] = 1    
#                 else:
#                     print('not in dictionary sense is : ', y_allSenses_Sent_batch[i][j][k])
            
    return mask_lemma2sense_batch    

#x_Raw_Sent_batch = x_Raw_Sent[:12]
#y_allSenses_Sent_batch = y_allSenses_Sent[:12]
#y_allSenses_Sent_id_batch = y_allSenses_Sent_id[:12]
#
#mask_lemma2sense_batch = mask_lemmaToSenses_batch(x_Raw_Sent_batch, y_allSenses_Sent_batch, y_allSenses_Sent_id_batch, SenseWords2id)    

# use pretrained embeddings function, for chars we are using pretrain file and for bigram, trigram 
# and fourgram we are using mean of embeddings of all unigrams of them
def reading_pretrained_sense_embeddings(filename, RawWords2id):
    # Reading Pretrained Embeddings from file
    pretrain_embeddigs = {}
    with codecs.open(filename, "r", "utf-8") as f:
        for line in f:
            pre_train = line.split()
            if len(pre_train) > 2:
                word = pre_train[0]
                if word in RawWords2id:
                    vec = pre_train[1:]
                    pretrain_embeddigs[word] = vec                
    
    print("pretrained embeddings files reading finished ...")
    # making embeddings for all RawWords2id.
    embedding_dim = len(next(iter(pretrain_embeddigs.values())))
    out_of_vocab = 0
    out = np.ones((len(RawWords2id), embedding_dim))*0.001
    for word in RawWords2id.keys():
        if len(word) == 1:
            if word in pretrain_embeddigs.keys():        
                out[RawWords2id[word]]=np.array(pretrain_embeddigs[word])
            else:                
                out_of_vocab+=1
                np.random.uniform(-1.0, 1.0, embedding_dim)
        
    return out,out_of_vocab


In [0]:

PAD_ID = RawWords2id[PAD]
id2Sensewords = dict(zip(SenseWords2id.values(), SenseWords2id.keys()))

sense_embeddings, out_of_vocab = reading_pretrained_sense_embeddings(root_path + '/allWordsSensesembeddings.vec', RawWords2id)
print('sense_embeddings: ', len(sense_embeddings))
print('out_of_vocab: ', out_of_vocab)

# we are using pretrained embedding - len(words2id)
VOCAB_SIZE =  len(RawWords2id)
WORD_EMBEDDING_DIM = 100
print('WORD_EMBEDDING_DIM: ', WORD_EMBEDDING_DIM)
print('VOCAB_SIZE: ', VOCAB_SIZE)

# some Basic Hyper parameters
NUM_EPOCHS = 5
BATCH_SIZE = 4
HIDDEN_LAYER_DIM = 256
LEARNING_RATE = 1
NUM_CLASSES = len(SenseWords2id)
print('num sense classes: ', NUM_CLASSES)
L2_REGU_LAMBDA=0.0001
NUM_LAYERS = 2
CLIP=10
crf_lambda = 0.05
num_sense_pos = len(pos2id)
print('num_sense_pos: ', num_sense_pos)
num_sense_lex = len(lex2id)
print('num_sense_lex: ', num_sense_lex)
num_sense_wnDomain = len(wnDomain2id)
print('num_sense_wnDomain: ', num_sense_wnDomain)
init_lr = 0.005
decay_steps = 500
decay_rate = 0.96
moving_avg_deacy = 0.999
VERY_BIG_NUMBER = 1e30

summaries = []




pretrained embeddings files reading finished ...
sense_embeddings:  32893
out_of_vocab:  5
WORD_EMBEDDING_DIM:  100
VOCAB_SIZE:  32893
num sense classes:  60919
num_sense_pos:  12
num_sense_lex:  46
num_sense_wnDomain:  160


In [0]:

# --------------------- Tensorflow part ---------------------------------------------------
import tensorflow as tf
from tensorflow.contrib import layers
from tensorflow.contrib import crf

def create_tensorflow_model(vocab_size, embedding_dim, hidden_layer_dim, sense_embed_bool = False, total_loss = 0):
    print("Creating TENSORFLOW model")
     
    # Inputs have (batch_size, timesteps) shape.
    input_ = tf.placeholder(dtype = tf.int32, shape=[None, None], name='input_x')   
    # Labels have (batch_size,) shape.
    labels = tf.placeholder(dtype = tf.int32, shape=[None, None], name='labels_BN')
    
    y_pos = tf.placeholder(dtype = tf.int32, shape=[None, None], name='pos_tag')
    
    y_lex = tf.placeholder(dtype = tf.int32, shape=[None, None], name='lex_tag')
           
    
    # dropout_keep_prob is a scalar.
    dropout_keep_prob = tf.placeholder(dtype = tf.float32, name='dropout_keep_prob')
    
    seq_length = tf.reduce_sum(tf.cast( tf.not_equal(input_[:,:], tf.ones_like(input_[:,:] ) * PAD_ID ), tf.int32), 1)    
    print('seq_length: ', seq_length)
    
    max_sent_size = tf.size(input_[0,:])    
    print('max_sent_size: ', max_sent_size)
    
    batch_size = tf.size(input_[:,0])    
    print('batch_size: ', batch_size)        
    
    mask_RawTosense_batch = tf.placeholder(dtype = tf.bool, shape=[None, None, None], name='mask_Raw2Senses')    
    print('mask_RawTosense_batch: ', mask_RawTosense_batch)
    
    mask_pos2sense = tf.placeholder(dtype = tf.bool, shape=[None, None], name='mask_Raw2Senses')    
    print('mask_pos2sense: ', mask_pos2sense)
    mask_matrix_pos = tf.tile(tf.expand_dims(mask_pos2sense, 0), [batch_size, 1, 1])     
    print('mask_matrix_pos: ', mask_matrix_pos)
    
    mask_lex2sense = tf.placeholder(dtype = tf.bool, shape=[None, None], name='mask_Raw2Senses')    
    print('mask_lex2sense: ', mask_lex2sense)
    mask_matrix_lex = tf.tile(tf.expand_dims(mask_lex2sense, 0), [batch_size, 1, 1])     
    print('mask_matrix_lex: ', mask_matrix_lex)
    
    
    x_mask = tf.not_equal(input_[:,:], tf.ones_like(input_[:,:] ) * PAD_ID )    
    print('x_mask: ', x_mask)
    
    attention_mask = (tf.cast(x_mask, 'float') -1) * VERY_BIG_NUMBER     
    print('attention_mask: ', attention_mask)

    # initialize weights randomly from a Gaussian distribution
    # step 1: create the initializer for weights
    weight_initer = tf.truncated_normal_initializer(mean=0.0, stddev=0.01)
        
## ---------------------------- embedding Block --------------------------------------------
## -----------------------------------------------------------------------------------------  
    with tf.variable_scope('embeddings', reuse=tf.AUTO_REUSE):
        if sense_embed_bool:
            embedding_matrix = tf.Variable(sense_embeddings, dtype=tf.float32, name='embedding')
        else:
            embedding_matrix = tf.get_variable("embeddings", shape=[vocab_size, embedding_dim])
        
        embeddings = tf.nn.embedding_lookup(embedding_matrix, input_)        
        print('embeddings: ', embeddings)
        
        embeddings = tf.reshape(embeddings,[batch_size, max_sent_size, embedding_dim]) #         
        print('embeddings: ', embeddings)

        embeddings=tf.nn.dropout(tf.cast(embeddings, tf.float32), dropout_keep_prob) #     embeddings shape (batch size, sentence length with padding, 100) 

        print ('embeddings is ok \n')                
    
## --------------------------- POS Block ---------------------------------------------------
## -----------------------------------------------------------------------------------------  

    with tf.variable_scope('rnn_cell_pos', reuse=tf.AUTO_REUSE):
            print ('lstm_cell_pos is is ok ... ')
            def lstm_cell1():
                return tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(hidden_layer_dim), output_keep_prob=dropout_keep_prob)

            stacked_fw_lstm_pos = tf.nn.rnn_cell.MultiRNNCell(
                [lstm_cell1() for _ in range(NUM_LAYERS)])

            stacked_bw_lstm_pos = tf.nn.rnn_cell.MultiRNNCell(
                [lstm_cell1() for _ in range(NUM_LAYERS)])

    with tf.variable_scope('rnn_pos', reuse=tf.AUTO_REUSE):                        
        (forward_output_pos, backword_output_pos), _ = tf.nn.bidirectional_dynamic_rnn(
            cell_fw = stacked_fw_lstm_pos,
            cell_bw = stacked_bw_lstm_pos,
            inputs = embeddings,
            sequence_length = seq_length,
            dtype=tf.float32
        )                        
        outputBD_pos = tf.concat([forward_output_pos, backword_output_pos], axis=2)
        print('outputBD_pos: ', outputBD_pos) # shape is batch*max_len*(2*hidden_layer_size)                         

        print ('outputBD_pos is ok \n')

    with tf.variable_scope("softmax_layer_pos", reuse=tf.AUTO_REUSE):
        W_pos = tf.get_variable("W_pos", shape=[2*hidden_layer_dim, num_sense_pos], initializer = weight_initer)
        b_pos = tf.get_variable("b_pos", shape=[num_sense_pos], initializer=tf.zeros_initializer())

        flat_softmax_pos = tf.reshape(outputBD_pos, [-1, tf.shape(outputBD_pos)[2]]) # shape is (batch*max_len)*(2*hidden_layer_size)
        print('flat_softmax_pos: ', flat_softmax_pos)

        drop_flat_softmax_pos = tf.nn.dropout(flat_softmax_pos, dropout_keep_prob) # shape is (batch*max_len)*(2*hidden_layer_size)
        print('drop_flat_softmax_pos: ', drop_flat_softmax_pos)

        flat_pos_logits = tf.matmul(drop_flat_softmax_pos, W_pos) + b_pos # shape is (batch*max_len)*num_sense_pos
        print('flat_pos_logits: ', flat_pos_logits)

        pos_logits = tf.reshape(flat_pos_logits, [batch_size, max_sent_size, num_sense_pos]) # shape is batch*max_len*num_sense_pos ->3D
        print('pos_logits: ', pos_logits)

        predict_pos = tf.argmax(pos_logits , axis=2)

        loss_pos = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels= y_pos, logits= pos_logits))

        total_loss = total_loss + loss_pos

        float_mask_matrix_pos = tf.cast(mask_matrix_pos, dtype=tf.float32)
        print('float_mask_matrix_pos: ', float_mask_matrix_pos)

        pos_masked_senses = tf.matmul(pos_logits, float_mask_matrix_pos) #mask_mat = 2*11*56 or batch*pos_num*NUM_CLASSES
        print('pos_masked_senses: ', pos_masked_senses)  # size is: batch*max_len*NUM_CLASSES

        print ('softmax_layer_pos is ok \n')


## --------------------------- LEX Block ---------------------------------------------------
## -----------------------------------------------------------------------------------------
    with tf.variable_scope('rnn_cell_lex', reuse=tf.AUTO_REUSE):
        print ('lstm_cell_lex is is ok ... ')
        def lstm_cell1():
            return tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(hidden_layer_dim), output_keep_prob=dropout_keep_prob)

        stacked_fw_lstm_lex = tf.nn.rnn_cell.MultiRNNCell(
            [lstm_cell1() for _ in range(NUM_LAYERS)])

        stacked_bw_lstm_lex = tf.nn.rnn_cell.MultiRNNCell(
            [lstm_cell1() for _ in range(NUM_LAYERS)])

    with tf.variable_scope('rnn_lex', reuse=tf.AUTO_REUSE):                        
        (forward_output_lex, backword_output_lex), _ = tf.nn.bidirectional_dynamic_rnn(
            cell_fw = stacked_fw_lstm_lex,
            cell_bw = stacked_bw_lstm_lex,
            inputs = outputBD_pos,
            sequence_length = seq_length,
            dtype=tf.float32
        )

        outputBD_lex = tf.concat([forward_output_lex, backword_output_lex], axis=2)
        print('outputBD_lex: ', outputBD_lex) # shape is batch*max_len*(2*hidden_layer_size) 

        print ('outputBD_lex is ok \n')     

    with tf.variable_scope("softmax_layer_lex", reuse=tf.AUTO_REUSE):
        W_lex = tf.get_variable("W_lex", shape=[2*hidden_layer_dim, num_sense_lex], initializer = weight_initer)
        b_lex = tf.get_variable("b_lex", shape=[num_sense_lex], initializer=tf.zeros_initializer())

        flat_softmax_lex = tf.reshape(outputBD_lex, [-1, tf.shape(outputBD_lex)[2]]) # shape is (batch*max_len)*(2*hidden_layer_size)
        print('flat_softmax_lex: ', flat_softmax_lex)

        drop_flat_softmax_lex = tf.nn.dropout(flat_softmax_lex, dropout_keep_prob) # shape is (batch*max_len)*(2*hidden_layer_size)
        print('drop_flat_softmax_lex: ', drop_flat_softmax_lex)

        flat_lex_logits = tf.matmul(drop_flat_softmax_lex, W_lex) + b_lex # shape is (batch*max_len)*num_sense_lex
        print('flat_lex_logits: ', flat_lex_logits)

        lex_logits = tf.reshape(flat_lex_logits, [batch_size, max_sent_size, num_sense_lex]) # shape is batch*max_len*num_sense_lex ->3D
        print('lex_logits: ', lex_logits)

        predict_lex = tf.argmax(lex_logits , axis=2)

        loss_lex = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels= y_lex, logits= lex_logits))
        total_loss = total_loss + loss_lex

        float_mask_matrix_lex = tf.cast(mask_matrix_lex, dtype=tf.float32)
        print('float_mask_matrix_lex: ', float_mask_matrix_lex)

        lex_masked_senses = tf.matmul(lex_logits, float_mask_matrix_lex)
        print('lex_masked_senses: ', lex_masked_senses)

        print ('softmax_layer_lex is ok \n')     

    input_rnn_sense = outputBD_lex

## --------------------------- SENSE Block ------------------------------------------------   
## ----------------------------------------------------------------------------------------   
    with tf.variable_scope('rnn_cell_sense', reuse=tf.AUTO_REUSE):
            print ('lstm_sense is ok')
            def lstm_cell():
                return tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(hidden_layer_dim), output_keep_prob=dropout_keep_prob)
        
            stacked_fw_lstm_Sense = tf.nn.rnn_cell.MultiRNNCell(
                [lstm_cell() for _ in range(NUM_LAYERS)])
            
            stacked_bw_lstm_Sense = tf.nn.rnn_cell.MultiRNNCell(
                [lstm_cell() for _ in range(NUM_LAYERS)])
            
    with tf.variable_scope('rnn_sense', reuse=tf.AUTO_REUSE):                        
            (forward_output_Sense, backword_output_Sense), _ = tf.nn.bidirectional_dynamic_rnn(
                cell_fw = stacked_fw_lstm_Sense,
                cell_bw = stacked_bw_lstm_Sense,
                inputs = input_rnn_sense,
                sequence_length = seq_length,
                dtype=tf.float32
            )
            
            outputBD_Sense = tf.concat([forward_output_Sense, backword_output_Sense], axis=2)            
            print('outputBD_Sense: ', outputBD_Sense) # shape is batch*max_len*(2*hidden_layer_size) 
            
            print ('outputBD_Sense is ok \n')
            
## --------------------------- attention_layer --------------------------------------------------
    with tf.variable_scope("attention_layer", reuse=tf.AUTO_REUSE):
        W_attention_L = tf.get_variable("W_attention_L", shape=[2*hidden_layer_dim, 1], initializer = weight_initer )
        flat_outputBD_Sense = tf.reshape(outputBD_Sense, [batch_size*max_sent_size, tf.shape(outputBD_Sense)[2]])        
        print('flat_outputBD_Sense: ', flat_outputBD_Sense) # shape is  shape is (batch*max_length*(2*hidden_layer_size)).(51(2*hidden_layer_size)2*1) = (batch*max_length)*1 ->2D
        
        flat_outputBD_Sense_tanh = tf.tanh(flat_outputBD_Sense)
        
        u_flat_outputBD_Sense_tanh = tf.matmul(flat_outputBD_Sense_tanh, W_attention_L) # shape is batch*max_length -> 2D
        print('u_flat: ', u_flat_outputBD_Sense_tanh)
        
        u = tf.reshape(u_flat_outputBD_Sense_tanh, [batch_size, max_sent_size]) + attention_mask
        print('u: ', u)
        
        u_softmax = tf.nn.softmax(u, 1)
        
        a = tf.expand_dims(u_softmax, 2) # shape is expand to batch*max_len*1 -> 3D
        print('a: ', a)
        
        c = tf.reduce_sum(tf.multiply(outputBD_Sense, a), axis=1) # shape is batch*max_len*(2*hidden_layer_size) and then reduce_sum with axis 1 to: batch*(2*hidden_layer_size)
        print('c: ', c)
        
        tiled_c = tf.tile(tf.expand_dims(c, 1), [1, max_sent_size, 1]) # shape is batch*max_len*(2*hidden_layer_size)
        print('tiled_c: ', tiled_c)
        
        attention_output = tf.concat([tiled_c, outputBD_Sense], 2) # batch*max_len*(2*hidden_layer_size) "concat with" batch*max_len*(2*hidden_layer_size) -> batch*max_len*(4*hidden_layer_size)
        print('attention_output: ', attention_output)
        
        flat_attention_output = tf.reshape(attention_output, [batch_size*max_sent_size, tf.shape(attention_output)[2]]) # reshape to (batch*max_len)*(4*hidden_layer_size) -> 2D
        print('flat_attention_output: ', flat_attention_output)

        print ('global_attention is ok \n') 
        
## --------------------------- hidden_layer Sense ------------------------------------------------------------
    with tf.variable_scope("hidden_layer", reuse=tf.AUTO_REUSE):
        W = tf.get_variable("W", shape=[4*hidden_layer_dim, 2*hidden_layer_dim], initializer = weight_initer)
        b = tf.get_variable("b", shape=[2*hidden_layer_dim], initializer = tf.zeros_initializer())
        
        drop_flat_attention_output = tf.nn.dropout(flat_attention_output, dropout_keep_prob) # shape is (batch*max_len)*(4*hidden_layer_size) -> 2D
        print('drop_flat_attention_output: ', drop_flat_attention_output)
        
        hidden_layer_output = tf.matmul(drop_flat_attention_output, W) + b  # shape is (batch*max_len)*(2*hidden_layer_size) -> 2D
        print('hidden_layer_output: ', hidden_layer_output)
                
        print ('hidden_layer is ok \n') 
        
## --------------------------- softmax_layer Sense ------------------------------------------------------------
    with tf.variable_scope("softmax_layer", reuse=tf.AUTO_REUSE):
        W_sense = tf.get_variable("W_sense", shape=[2*hidden_layer_dim, NUM_CLASSES], initializer = weight_initer)
        b_sense = tf.get_variable("b_sense", shape=[NUM_CLASSES], initializer=tf.zeros_initializer())
        
        drop_hidden_layer_output = tf.nn.dropout(hidden_layer_output, dropout_keep_prob)
        print('drop_hidden_layer_output: ', drop_hidden_layer_output)
        
        flat_sense_logits = tf.matmul(drop_hidden_layer_output, W_sense) + b_sense
        print('flat_sense_logits: ', flat_sense_logits)
        
        sense_logits = tf.reshape(flat_sense_logits, [batch_size, max_sent_size, NUM_CLASSES])
        print('sense_logits: ', sense_logits)                
        
        # we add combination of lex and pos masks to our logit to make a better filter of outputs to our sense logit.
        masked_sense_logits = tf.multiply(sense_logits, tf.multiply(pos_masked_senses, lex_masked_senses))
        print('masked_sense_logits: ', masked_sense_logits)  

            
        float_mask_RawTosense_batch = tf.cast(mask_RawTosense_batch, dtype=tf.float32)
        print('float_mask_RawTosense_batch: ', float_mask_RawTosense_batch)
        
        final_sense_logits = tf.multiply(masked_sense_logits, float_mask_RawTosense_batch)
        print('final_sense_logits: ', final_sense_logits)

        predictions = tf.argmax(final_sense_logits , axis=2)
        print('predictions: ', predictions)
        loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels= labels, logits= final_sense_logits))
        total_loss = total_loss + loss
        print ('softmax_layer is ok \n')
        
                      
## --------------------------- train_op Block ----------------------------------------------
## -----------------------------------------------------------------------------------------          
    with tf.variable_scope('train_op', reuse=tf.AUTO_REUSE):
                        
        optimizer = tf.train.AdadeltaOptimizer(LEARNING_RATE)
        
#         optimizer = tf.train.AdamOptimizer(LEARNING_RATE)
#         optimizer=tf.train.GradientDescentOptimizer(learning_rate=LEARNING_RATE, use_locking=False, name='GradientDescent')
        print ('AdamOptimizer is ok .... \n')
        
        tvars=tf.trainable_variables()
        print ('tvars is ok ....')
    
        l2_loss = tf.add_n([tf.nn.l2_loss(v) for v in tvars if v.get_shape().ndims > 1])
        print ('l2_loss is ok ....')
        
        total_loss = total_loss + L2_REGU_LAMBDA*l2_loss             
        print ('total_loss is ok ....')
        
        summaries.append(tf.summary.scalar("loss", loss))
        summaries.append(tf.summary.scalar("total_loss", total_loss))
        
        grads,_ = tf.clip_by_global_norm(tf.gradients(total_loss,tvars),CLIP)
        print ('grads is ok ....')
        
        train_op = optimizer.apply_gradients(zip(grads,tvars))
        print ('train_op apply_gradients is ok ....')
                               
    return input_, labels, y_pos, y_lex, mask_RawTosense_batch, train_op, predictions, dropout_keep_prob, total_loss, seq_length, mask_pos2sense, mask_lex2sense


In [0]:
# create tensorflow model without sense embedding and just basic LSTM
input_, labels, y_pos, y_lex, mask_RawTosense_batch, train_op, predictions, dropout_keep_prob, total_loss, seq_length, mask_pos2sense, mask_lex2sense\
= create_tensorflow_model(VOCAB_SIZE, WORD_EMBEDDING_DIM, HIDDEN_LAYER_DIM, sense_embed_bool = True, total_loss = 0)





Creating TENSORFLOW model
seq_length:  Tensor("Sum:0", shape=(?,), dtype=int32)
max_sent_size:  Tensor("Size:0", shape=(), dtype=int32)
batch_size:  Tensor("Size_1:0", shape=(), dtype=int32)
mask_RawTosense_batch:  Tensor("mask_Raw2Senses:0", shape=(?, ?, ?), dtype=bool)
mask_pos2sense:  Tensor("mask_Raw2Senses_1:0", shape=(?, ?), dtype=bool)
mask_matrix_pos:  Tensor("Tile:0", shape=(?, ?, ?), dtype=bool)
mask_lex2sense:  Tensor("mask_Raw2Senses_2:0", shape=(?, ?), dtype=bool)
mask_matrix_lex:  Tensor("Tile_1:0", shape=(?, ?, ?), dtype=bool)
x_mask:  Tensor("NotEqual_1:0", shape=(?, ?), dtype=bool)
attention_mask:  Tensor("mul_2:0", shape=(?, ?), dtype=float32)
embeddings:  Tensor("embeddings/embedding_lookup/Identity:0", shape=(?, ?, 100), dtype=float32)
embeddings:  Tensor("embeddings/Reshape:0", shape=(?, ?, 100), dtype=float32)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
embeddings is ok 

lstm_cell_pos is is ok

In [0]:
# Train and Dev split
# from sklearn.model_selection import train_test_split
# train_x_Raw, dev_x_Raw, train_y_BN, dev_y_BN, train_y_POS, dev_y_POS, train_y_allSenses_Sent, dev_y_allSenses_Sent, train_y_allSenses_Sent_id, dev_y_allSenses_Sent_id =\
#     train_test_split(x_train_Raw_Sent_id, y_train_BNsense_Sent_id, y_train_POS_Sent_id, y_train_allSenses_Sent, y_train_allSenses_Sent_id, test_size=0.01, random_state=42)


# subset_train = 50 # Put number of subset data you want for train
# subset_dev= 50
# subset_test= 50


train_path =  root_path + '/semcor/semcor/'
dev_path = root_path + '/semcor/semeval2007/'
test_path = root_path + '/semcor/semeval2013/'


x_train_Raw_Sent_id = pickle.load(open(train_path + 'x_Raw_Sent_id.pkl', "rb"))
y_train_BNsense_Sent_id = pickle.load(open(train_path + 'y_BNsense_Sent_id.pkl', "rb"))
y_train_POS_Sent_id = pickle.load(open(train_path + 'y_POS_Sent_id.pkl', "rb"))
y_train_LEX_Sent_id = pickle.load(open(train_path + 'y_Lex_Sent_id.pkl', "rb"))
y_train_allSenses_Sent = pickle.load(open(train_path + 'y_allSenses_Sent.pkl', "rb"))
y_train_allSenses_Sent_id = pickle.load(open(train_path + 'y_allSenses_Sent_id.pkl', "rb"))

mask_train_lex2sense = pickle.load(open(train_path + 'mask_lex2sense.pkl', "rb"))
mask_train_pos2sense = pickle.load(open(train_path + 'mask_pos2sense.pkl', "rb"))

# x_train_Raw_Sent_id = x_train_Raw_Sent_id[slice(0, subset_train)]
# y_train_BNsense_Sent_id = y_train_BNsense_Sent_id[slice(0, subset_train)]  
# y_train_POS_Sent_id = y_train_POS_Sent_id[slice(0, subset_train)]
# y_train_LEX_Sent_id = y_train_LEX_Sent_id[slice(0, subset_train)]  
# y_train_allSenses_Sent = y_train_allSenses_Sent[slice(0, subset_train)]
# y_train_allSenses_Sent_id = y_train_allSenses_Sent_id[slice(0, subset_train)]

print(len(x_train_Raw_Sent_id))
print(len(x_train_Raw_Sent_id[0]))
print(len(y_train_BNsense_Sent_id[0]))

print('train data is ok ......... ')

x_dev_Raw_Sent = pickle.load(open(dev_path + 'x_Raw_Sent.pkl', "rb"))
x_dev_Raw_Sent_id = pickle.load(open(dev_path + 'x_Raw_Sent_id.pkl', "rb"))
y_dev_BNsense_Sent_id = pickle.load(open(dev_path + 'y_BNsense_Sent_id.pkl', "rb"))
y_dev_POS_Sent_id = pickle.load(open(dev_path + 'y_POS_Sent_id.pkl', "rb"))
y_dev_LEX_Sent_id = pickle.load(open(dev_path + 'y_Lex_Sent_id.pkl', "rb"))
y_dev_allSenses_Sent = pickle.load(open(dev_path + 'y_allSenses_Sent.pkl', "rb"))
y_dev_allSenses_Sent_id = pickle.load(open(dev_path + 'y_allSenses_Sent_id.pkl', "rb"))

mask_dev_lex2sense = pickle.load(open(dev_path + 'mask_lex2sense.pkl', "rb"))
mask_dev_pos2sense = pickle.load(open(dev_path + 'mask_pos2sense.pkl', "rb"))
 
# x_dev_Raw_Sent_id = x_dev_Raw_Sent_id[slice(0, subset_dev)]
# y_dev_BNsense_Sent_id = y_dev_BNsense_Sent_id[slice(0, subset_dev)]  
# y_dev_POS_Sent_id = y_dev_POS_Sent_id[slice(0, subset_dev)]
# y_dev_LEX_Sent_id = y_dev_LEX_Sent_id[slice(0, subset_dev)]  
# y_dev_allSenses_Sent = y_dev_allSenses_Sent[slice(0, subset_dev)]
# y_dev_allSenses_Sent_id = y_dev_allSenses_Sent_id[slice(0, subset_dev)]

print(len(x_dev_Raw_Sent_id))
print(len(x_dev_Raw_Sent_id[0]))
print(len(y_dev_BNsense_Sent_id[0]))

print('dev data is ok ......... ')
    
x_test_Raw_Sent = pickle.load(open(test_path + 'x_Raw_Sent.pkl', "rb"))
x_test_Raw_Sent_id = pickle.load(open(test_path + 'x_Raw_Sent_id.pkl', "rb"))
y_test_BNsense_Sent_id = pickle.load(open(test_path + 'y_BNsense_Sent_id.pkl', "rb"))
y_test_POS_Sent_id = pickle.load(open(test_path + 'y_POS_Sent_id.pkl', "rb"))
y_test_allSenses_Sent = pickle.load(open(test_path + 'y_allSenses_Sent.pkl', "rb"))
y_test_allSenses_Sent_id = pickle.load(open(test_path + 'y_allSenses_Sent_id.pkl', "rb"))

mask_test_lex2sense = pickle.load(open(test_path + 'mask_lex2sense.pkl', "rb"))
mask_test_pos2sense = pickle.load(open(test_path + 'mask_pos2sense.pkl', "rb"))
 
    
# x_test_Raw_Sent_id = x_test_Raw_Sent_id[slice(0, subset_test)]
# y_test_BNsense_Sent_id = y_test_BNsense_Sent_id[slice(0, subset_test)]  
# y_test_POS_Sent_id = y_test_POS_Sent_id[slice(0, subset_test)]
# y_test_allSenses_Sent = y_test_allSenses_Sent[slice(0, subset_test)]
# y_test_allSenses_Sent_id = y_test_allSenses_Sent_id[slice(0, subset_test)]

print(len(x_test_Raw_Sent_id))
print(len(x_test_Raw_Sent_id[0]))
print(len(y_test_BNsense_Sent_id[0]))

print('test data is ok ......... ')



37176
17
17
train data is ok ......... 
135
36
36
dev data is ok ......... 
306
7
7
test data is ok ......... 


In [0]:
SUBSET_MODEL_ADD = root_path+'/POS_LEX_BDLSTM_SenseEmbed_models_47.40406320541761/.'
print(SUBSET_MODEL_ADD)


/content/gdrive/My Drive/NLPHW3/POS_LEX_BDLSTM_SenseEmbed_models_47.40406320541761/.


In [0]:

# -----------------------------------------------------------------------------
def WN_to_BN_dic(mapping_file):
    print('WN_to_BN_dic is started ....')
    BN_ID = []
    WN_ID = []
    with codecs.open(mapping_file,'rb') as f:        
        for line in f:            
            line_synsets = line.decode().strip().split('\t')
            BN_ID.append(line_synsets[0])
            WN_ID.append(line_synsets[1])    
    WN2BN_map_dic = dict(zip(WN_ID,BN_ID))
    print('WN_to_BN_dic is done ....')

    return WN2BN_map_dic


WN2BN_map_dic = WN_to_BN_dic(MAP_WN2BN_FILE)
print(len(WN2BN_map_dic))

## ----------------------------------------------------------------------------
def getMFS_(word, WN2BN_map_dic):
#     print('no prediction for: ', word)
    all_synsets = wn.synsets(word)

    # check if the word has synsets or not!
    if len(all_synsets) == 0 or all_synsets is None:
        return word
    
    synset = all_synsets[0]
    MFS_wnId = "wn:" + str(synset.offset()).zfill( 8) + synset.pos()
    if WN2BN_map_dic.get(MFS_wnId) is not None:
        MFS_bnId = WN2BN_map_dic.get(MFS_wnId)
        
    return MFS_bnId


WN_to_BN_dic is started ....
WN_to_BN_dic is done ....
117659


In [0]:
# Run tensorflow model
saver = tf.train.Saver()


with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess: 
    # I couln't use tensorboard because it causes my colab session to crash for big datasets.
    #train_writer = tf.summary.FileWriter(root_path+'logging/tensorflow_model', sess.graph)
    
    if SUBSET_MODEL_ADD is not None:
        saver = tf.train.Saver()
        saver.restore(sess, tf.train.latest_checkpoint(SUBSET_MODEL_ADD))
        print('model restored')
    else:
        sess.run(tf.global_variables_initializer())             
        print('global variable initialized')
        
    print ('start session is ok ....')
        
    for epoch in range(NUM_EPOCHS):        
        #train
        train_loss=[]    
        for i in range(0, len(x_train_Raw_Sent_id), BATCH_SIZE):
            # slice dataset for padding
                        
            batch_x_Raw = x_train_Raw_Sent_id[slice(i, i + BATCH_SIZE)]
            batch_y_BN = y_train_BNsense_Sent_id[slice(i, i + BATCH_SIZE)]
            batch_y_POS = y_train_POS_Sent_id[slice(i, i + BATCH_SIZE)]            
            batch_y_LEX = y_train_LEX_Sent_id[slice(i, i + BATCH_SIZE)]            
            y_train_allSenses_Sent_batch = y_train_allSenses_Sent[slice(i, i + BATCH_SIZE)]
            y_train_allSenses_Sent_id_batch = y_train_allSenses_Sent_id[slice(i, i + BATCH_SIZE)]
            mask_lemma2Sense_b = mask_lemmaToSenses_batch(batch_x_Raw, y_train_allSenses_Sent_batch, y_train_allSenses_Sent_id_batch, SenseWords2id)
            
            
            batch_x_Raw = padding3(batch_x_Raw, PAD_ID)
            batch_y_BN = padding(batch_y_BN, PAD_ID)
            batch_y_POS = padding(batch_y_POS, PAD_ID)            
            batch_y_LEX = padding(batch_y_LEX, PAD_ID)
            
            # runnig training session ang getting train loss for every batch and append in epoch loss
            _, loss_other = sess.run(
                [train_op, total_loss], feed_dict = { input_ : batch_x_Raw, labels : batch_y_BN, y_pos : batch_y_POS, y_lex : batch_y_LEX,\
                                                     mask_RawTosense_batch : mask_lemma2Sense_b, mask_pos2sense: mask_train_pos2sense ,\
                                                     mask_lex2sense : mask_train_lex2sense,  dropout_keep_prob : 0.3 })
            
            train_loss.append(loss_other)            
            
            if i % (BATCH_SIZE * 200) == 0:                
                print ('Train batch %d loss %f' % (i, loss_other))
                
        train_loss=np.mean(train_loss,dtype=float)                
        print ('Train Epoch %d loss %f' % (epoch+1, train_loss))        
        print('--------------------------------') 
        
                                       
#         #Dev
        dev_loss=[]
        dev_pred=[]
        dev_hit_all=0
        dev_all_char=0        
        for i in range(0, len(x_dev_Raw_Sent_id), BATCH_SIZE):            
            
            batch_x_Lemma = x_dev_Raw_Sent[slice(i, i + BATCH_SIZE)]
            batch_x_Raw = x_dev_Raw_Sent_id[slice(i, i + BATCH_SIZE)]
            batch_y_BN = y_dev_BNsense_Sent_id[slice(i, i + BATCH_SIZE)]
            batch_y_POS = y_dev_POS_Sent_id[slice(i, i + BATCH_SIZE)]            
            batch_y_LEX = y_dev_LEX_Sent_id[slice(i, i + BATCH_SIZE)]
            
            y_dev_allSenses_Sent_batch = y_dev_allSenses_Sent[slice(i, i + BATCH_SIZE)]
            y_dev_allSenses_Sent_id_batch = y_dev_allSenses_Sent_id[slice(i, i + BATCH_SIZE)]            
            mask_lemma2Sense_b = mask_lemmaToSenses_batch(batch_x_Raw, y_dev_allSenses_Sent_batch, y_dev_allSenses_Sent_id_batch, SenseWords2id)            
            
            batch_x_Raw = padding3(batch_x_Raw, PAD_ID)
            batch_y_BN = padding(batch_y_BN, PAD_ID)
            batch_y_POS = padding(batch_y_POS, PAD_ID)
            batch_y_LEX = padding(batch_y_LEX, PAD_ID)

            
            loss_other, lengths, predict = sess.run(
                [total_loss, seq_length, predictions], feed_dict = {input_ : batch_x_Raw, labels : batch_y_BN, y_pos : batch_y_POS, y_lex : batch_y_LEX,\
                                                                                        mask_RawTosense_batch : mask_lemma2Sense_b, mask_pos2sense: mask_dev_pos2sense ,\
                                                                                        mask_lex2sense : mask_dev_lex2sense, dropout_keep_prob: 1.0 })
            
            batch_true = y_dev_BNsense_Sent_id[slice(i, i + BATCH_SIZE)]
        
            pred_sense = []
            true_sense = []            
            for p in range(len(predict)):
                pr = []
                tr = []
                for L_ in range(lengths[p]):
                    # if our prediction is UNK means that word is not in our dictionary and we can't predict anything for that, so we put BackOff plan and MFS as our prediction.
                    if id2Sensewords[predict[p][L_]] != UNK:
                        pr.append(id2Sensewords[predict[p][L_]])
                    else:
                        pr.append(getMFS_(batch_x_Lemma[p][L_], WN2BN_map_dic))
#                         print(getMFS_(batch_x_Lemma[p][L_], WN2BN_map_dic))                        
                                                        
                    tr.append(id2Sensewords[batch_true[p][L_]])
                print(pr)
                print(tr)
                print('\n')
                pred_sense.append(pr)
                true_sense.append(tr)
                        
            batch_pred = []
            for p in range(len(predict)):                
                dev_pred.append(predict[p][:lengths[p]])
                batch_pred.append(predict[p][:lengths[p]])
                
#             print('\n')
            dev_batch_hit, dev_batch_char = number_of_batch_sense_hits(batch_pred, batch_true)            
            print('dev batch %d loss %f dev_batch_hit:%d dev_batch_char:%d batch_accuracy:%f' % (i, loss_other, dev_batch_hit, dev_batch_char, (dev_batch_hit/dev_batch_char)*100 ))
            dev_loss.append(loss_other)            
            dev_hit_all+=dev_batch_hit
            dev_all_char+=dev_batch_char
                
        dev_loss=np.mean(dev_loss,dtype=float)
        dev_acc = (dev_hit_all/dev_all_char)*100
        print('Valid Epoch %d loss %f' % (epoch+1,dev_loss))        
        print('dev_hit_all:%d dev_all_char:%d accuracy:%f' % (dev_hit_all,dev_all_char,dev_acc ))
        print('--------------------------------')

        
        save_path = saver.save(sess, root_path+'/POS_LEX_BDLSTM_SenseEmbed_models_{}/model.ckpt'.format(dev_acc))
        print("Model saved in path: %s" % save_path)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/NLPHW3/POS_LEX_BDLSTM_SenseEmbed_models_47.40406320541761/model.ckpt


NotFoundError: ignored